In [4]:
import numpy as np

In [5]:
class Expression:
    def __init__(self, ele_func, sub_expr1, sub_expr2=None):
        self._ele_func  = ele_func
        self._sub_expr1 = sub_expr1
        self._sub_expr2 = sub_expr2
    
    def evaluation_at(self, val_dict):
        
        # self._sub_expr2 is None implies that self._ele_func is an unary operator
        if self._sub_expr2 is None: 
            return self._ele_func.evaluation_at(
                self._sub_expr1, val_dict)
        
        # self._sub_expr2 not None implies that self._ele_func is a binary operator
        else:
            return self._ele_func.evaluation_at(
                self._sub_expr1, self._sub_expr2, val_dict)
    
    def derivative_at(self, var, val_dict):
        
        if var is self: return 1.0
        
        # sub_expr2 is None implies that _ele_func is an unary operator
        if self._sub_expr2 is None:
            return self._ele_func.derivative_at(
                self._sub_expr1, var, val_dict)
        
        # sub_expr2 not None implies that _ele_func is a binary operator
        else:
            return self._ele_func.derivative_at(
                self._sub_expr1, self._sub_expr2, var, val_dict)
    
    def __add__(self, another):
        if isinstance(another, Expression):
            return Expression(Add, self, another)
        # if the other operand is not an Expression, then it must be a number
        # the number then should be converted to a Constant
        else:
            return Expression(Add, self, Constant(another))
    
    def __radd__(self, another):
        if isinstance(another, Expression):
            return Expression(Add, another, self)
        else:
            return Expression(Add, Constant(another), self)
    
    def __sub__(self, another):
        if isinstance(another, Expression):
            return Expression(Sub, self, another)
        else:
            return Expression(Sub, self, Constant(another))
    
    def __rsub__(self, another):
        if isinstance(another, Expression):
            return Expression(Sub, another, self)
        else:
            return Expression(Sub, Constant(another), self)
        

    def __mul__(self, another):
        if isinstance(another, Expression):
            return Expression(Mul,self,another)
        else:
            return Expression(Mul, self, Constant(another))

    def __rmul__(self, another):
        if isinstance(another, Expression):
            return Expression(Mul,another,self)
        else:
            return Expression(Mul, Constant(another),self)
    
    def __truediv__(self, another):
        if isinstance(another, Expression):
            return Expression(Div,self,another)
        else:
            return Expression(Div, self, Constant(another))

    def __rtruediv__(self, another):
        if isinstance(another, Expression):
            return Expression(Div,another,self)
        else:
            return Expression(Div, Constant(another),self)

In [6]:
class Variable(Expression):
    def __init__(self):
        return
    
    def evaluation_at(self, val_dict):
        return val_dict[self]
    
    def derivative_at(self, var, val_dict):
        return 1.0 if var is self else 0.0

In [7]:
class Constant(Expression):
    def __init__(self, val):
        self.val = val
        
    def evaluation_at(self, val_dict):
        return self.val
    
    def derivative_at(self, var, val_dict):
        return 0.0

In [8]:
class Add:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) + \
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) + \
               sub_expr2.derivative_at(var, val_dict)

In [9]:
class Sub:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) - \
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) - \
               sub_expr2.derivative_at(var, val_dict)

In [10]:
class Mul:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) *\
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) * \
               sub_expr2.evaluation_at(val_dict)+ \
               sub_expr1.evaluation_at(val_dict) *\
               sub_expr2.derivative_at(var, val_dict)

In [11]:
class Div:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) /\
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return  sub_expr1.derivative_at(var, val_dict) / \
                sub_expr2.evaluation_at(val_dict)+ \
                sub_expr1.evaluation_at(val_dict) *\
                sub_expr2.derivative_at(var, val_dict)/\
                sub_expr2.evaluation_at(val_dict)/\
                sub_expr2.evaluation_at(val_dict)

In [12]:
class Exp:
    @staticmethod
    def evaluation_at(sub_expr1, val_dict):
        return np.exp(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) * np.exp(sub_expr1.evaluation_at(val_dict))

In [13]:
def exp(expr):
    return Expression(Exp, expr)

In [14]:
class Sin:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return np.sin(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        return sub_expr1.derivative_at(var, val_dict)*np.cos(sub_expr1.evaluation_at(val_dict)) 

In [15]:
def sin(expr):
    return Expression(Sin,expr)

In [16]:
a = Variable()
b = Variable()
c = a+b
f1 = sin(a+c)

In [17]:
f2 = sin(a*b)

In [18]:
f2.evaluation_at({a:1,b:2})

0.9092974268256817

In [19]:
f2.derivative_at(a,{a:2,b:2})

-1.3072872417272239

In [20]:
#expected derivative
np.cos(4)*2

-1.3072872417272239

In [21]:
# function output
## may need to improve the interface.
f1.evaluation_at({a:1, b: 2.0})

-0.7568024953079282

In [22]:
# expected value
np.sin(1+2+1)

-0.7568024953079282

In [23]:
f1.derivative_at(c,{a: 1.0, b: 1.0})

-0.9899924966004454

$$\frac{df}{dc} = cos(a+c) = cos(3)$$

In [24]:
np.cos(3)

-0.9899924966004454

Let $f(a, b, c) = e^{a-b+c}$, at $(a, b, c) = (1.0, 2.0, 3.0)$, that should be $e^{2} \approx 7.389$.

In [25]:
a = Variable()
b = Variable()
c = Variable()
f = exp(a-b+c)

In [26]:
f.evaluation_at({a: 1.0, b: 2.0, c: 3.0})

7.38905609893065

In [27]:
class Cos:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return np.cos(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        return -sub_expr1.derivative_at(var, val_dict)*np.sin(sub_expr1.evaluation_at(val_dict)) 

In [28]:
def cos(expr):
    return Expression(Cos,expr)

In [29]:
a = Variable()
b = Variable()
c = a+b
f1 = cos(a+c)
f2 = cos(a*b)

In [30]:
print(f1.evaluation_at({a:1.0, b: 2.0}))
print(f2.evaluation_at({a:1.0,b:2}))

-0.6536436208636119
-0.4161468365471424


In [31]:
# expected
print(np.cos(4))
print(np.cos(2))

-0.6536436208636119
-0.4161468365471424


In [32]:
f1.derivative_at(a,{a:1.0, b: 2.0})

1.5136049906158564

In [33]:
# expected
-np.sin(1+3)*2

1.5136049906158564

In [34]:
f2.derivative_at(a,{a:2,b:2})

1.5136049906158564

In [35]:
#expected
print(-np.sin(2*2)*2)

1.5136049906158564


In [36]:
class Tan:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return np.tan(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        return sub_expr1.derivative_at(var, val_dict)*(1/np.cos(2*sub_expr1.evaluation_at(val_dict)))
def tan(expr):
    return Expression(Tan,expr)

In [37]:
a = Variable()
b = Variable()
c = a*b
f = tan(c*b)

In [38]:
f.evaluation_at({a:1,b:2})

1.1578212823495775

In [39]:
#expected
np.tan(4)

1.1578212823495775

In [40]:
f.derivative_at(c,{a:1,b:2})

-13.74570127338074

In [41]:
# expected derivative
2*(1/np.cos(4*2))

-13.74570127338074

In [42]:
class Cotan:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return 1/np.tan(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
    
        return -sub_expr1.derivative_at(var, val_dict)*(1/np.sin(sub_expr1.evaluation_at(val_dict))**2)

def cotan(expr):
    return Expression(Cotan,expr)

In [43]:
a = Variable()
b = Variable()
c = a*b
f = cotan(c*b)

In [44]:
f.evaluation_at({a:1,b:2})

0.8636911544506167

In [45]:
1/np.tan(4)

0.8636911544506167

In [46]:
f.derivative_at(c,{a:1,b:2})

-3.491924820552478

In [47]:
-(1/(np.sin(4)**2))*2

-3.491924820552478

In [48]:
class Sec:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return 1/np.cos(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x=sub_expr1.evaluation_at(val_dict)
        return sub_expr1.derivative_at(var, val_dict)*np.tan(x)*(1/np.cos(x))
def sec(expr):
    return Expression(Sec,expr) 

In [49]:
a = Variable()
b = Variable()
c = a*b
f = sec(c*b)

In [50]:
print(f.evaluation_at({a:1,b:2}))
print(1/np.cos(4))

-1.5298856564663974
-1.5298856564663974


In [51]:
print(f.derivative_at(c,{a:1,b:2}))
print(np.tan(4)*(1/np.cos(4))*2)

-3.542668345236299
-3.542668345236299


In [52]:
class Csc:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return 1/np.sin(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x=sub_expr1.evaluation_at(val_dict)
        return sub_expr1.derivative_at(var, val_dict)*(1/np.tan(x))*(1/np.sin(x))
def csc(expr):
    return Expression(Csc,expr) 

In [53]:
a = Variable()
b = Variable()
c = a*b
f = csc(c*b)

In [54]:
print(f.evaluation_at({a:1,b:2}))
print(1/np.sin(4))

-1.3213487088109024
-1.3213487088109024


In [55]:
print(f.derivative_at(c,{a:1,b:2}))
print((1/np.tan(4))*(1/np.sin(4))*2)

-2.2824743834894403
-2.2824743834894403


In [56]:
class Sinh:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return np.sinh(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x=sub_expr1.evaluation_at(val_dict)
        return sub_expr1.derivative_at(var, val_dict)*np.cosh(x)
def sinh(expr):
    return Expression(Sinh,expr) 

In [57]:
a = Variable()
b = Variable()
c = a*b
f = sinh(c*b)

In [64]:
assert(f.evaluation_at({a:1,b:2})==np.sinh(4))

In [63]:
assert(f.derivative_at(c,{a:1,b:2})==np.cosh(4)*2)

In [66]:
class Cosh:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        return np.cosh(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x=sub_expr1.evaluation_at(val_dict)
        return sub_expr1.derivative_at(var, val_dict)*np.sinh(x)
def cosh(expr):
    return Expression(Cosh,expr) 

In [67]:
a = Variable()
b = Variable()
c = a*b
f = cosh(c*b)
assert(f.evaluation_at({a:3,b:2})==np.cosh(12))
assert(f.derivative_at(c,{a:3,b:2})==np.sinh(12)*2)

In [ ]:
class Tanh
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return np.sinh(x)/np.cosh(x)
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return sub_expr1.derivative_at(var, val_dict)*(1/np.cosh(x)**2)
def tanh(expr):
    return Expression(Tanh,expr) 

In [68]:
a = Variable()
b = Variable()
c = a*b
f = tan(c*b)
assert(f.evaluation_at({a:3,b:2})==np.sin(12)/np.cos(12))
assert(f.derivative_at(c,{a:3,b:2})==1/np.cos(24)*2)

In [65]:
class Csch:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return 1/np.sinh(x)
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        # -csch(x)*cot(x)
        d=-(1/np.sinh(x))*(np.cosh(x)/np.sinh(x))
        return sub_expr1.derivative_at(var, val_dict)*d
def csch(expr):
    return Expression(Csch,expr) 

In [72]:
a = Variable()
b = Variable()
c = a*b
f = csch(c*b)
print(f.evaluation_at({a:3,b:2}))
print(1/np.sinh(12))
print(f.derivative_at(c,{a:3,b:2}))
print(-(np.cosh(12)/np.sinh(12))*(1/np.sinh(12))*2)

1.2288424707120323e-05
1.2288424707120323e-05
-2.4576849416096264e-05
-2.4576849416096264e-05


In [75]:
class Sech:
    def evaluation_at(sub_expr1,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return 1/np.cosh(x)
    
    def derivative_at(sub_expr1,var,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        # -sech(x)tanh(x)
        d=-(1/np.cosh(x))*(np.sinh(x)/np.cosh(x))
        return sub_expr1.derivative_at(var, val_dict)*d
def sech(expr):
    return Expression(Sech,expr)     

In [79]:
a = Variable()
b = Variable()
c = a*b
f = sech(c*b)
print(f.evaluation_at({a:2,b:1}))
print(1/np.cosh(2))
print(f.derivative_at(c,{a:2,b:1}))
print(-(np.sinh(2)/np.cosh(2))*(1/np.cosh(2))*1)

0.2658022288340797
0.2658022288340797
-0.2562406794416764
-0.2562406794416764


In [81]:
class Coth:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return np.cosh(x)/np.sinh(x)
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        #-csch^2(x)
        return -sub_expr1.derivative_at(var, val_dict)*(1/np.sinh(x)**2)

def coth(expr):
    return Expression(Coth,expr)    

In [84]:
a = Variable()
b = Variable()
c = a*b
f = coth(c*b)
print(f.evaluation_at({a:3,b:2}))
print(np.cosh(12)/np.sinh(12))
print(f.derivative_at(c,{a:3,b:2}))
print(-(1/np.sinh(12))**2*2)

1.0000000000755027
1.0000000000755027
-3.020107635651304e-10
-3.020107635651304e-10


In [97]:
class Arcsin:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return np.arcsin(x)
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        d = 1/np.sqrt(1-x**2)
        #1/sqrt(1-x^2)
        return sub_expr1.derivative_at(var, val_dict)*d

def arcsin(expr):
    return Expression(Arcsin,expr)

In [99]:
a = Variable()
b = Variable()
c = a*b
f = arcsin(c*b)
print(f.evaluation_at({a:0.2,b:0.5}))
print(np.arcsin(0.05))
print(f.derivative_at(c,{a:0.2,b:0.5}))
print((1/np.sqrt(1-(0.2*0.5*0.5)**2))*0.5)

0.050020856805770016
0.050020856805770016
0.5006261743217588
0.5006261743217588


In [102]:
class Arccos:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return np.arccos(x)
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        d = 1/np.sqrt(1-x**2)
        #-1/sqrt(1-x^2)
        return -sub_expr1.derivative_at(var, val_dict)*d

def arccos(expr):
    return Expression(Arccos,expr)

In [103]:
a = Variable()
b = Variable()
c = a*b
f = arccos(c*b)
print(f.evaluation_at({a:0.2,b:0.5}))
print(np.arccos(0.05))
print(f.derivative_at(c,{a:0.2,b:0.5}))
print((-1/np.sqrt(1-(0.2*0.5*0.5)**2))*0.5)

1.5207754699891265
1.5207754699891265
-0.5006261743217588
-0.5006261743217588


In [111]:
class Arctan:
    @staticmethod
    def evaluation_at(sub_expr1,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        return np.arctan(x)
    
    @staticmethod
    def derivative_at(sub_expr1,var,val_dict):
        x = sub_expr1.evaluation_at(val_dict)
        d = 1/(1+x**2)
        #1/1-x^2
        return sub_expr1.derivative_at(var, val_dict)*d

def arctan(expr):
    return Expression(Arctan,expr)

In [112]:
a = Variable()
b = Variable()
c = a*b
f = arctan(c*b)
print(f.evaluation_at({a:2,b:3}))
print(np.arctan(18))
print(f.derivative_at(c,{a:2,b:3}))
print((1/(18**2+1))*3)

1.5152978215491797
1.5152978215491797
0.00923076923076923
0.00923076923076923


$\dfrac{\partial f}{\partial b} = -e^{a-b+c}$, at $(a, b, c) = (1.0, 2.0, 3.0)$, that should be $-e^{2} \approx -7.389$.

In [73]:
f.derivative_at(b, {a: 1.0, b: 2.0, c: 3.0})

-4.0
-1.1455000338086134


13.893553038339718

$\dfrac{\partial f}{\partial a} = e^{a-b+c}$, at $(a, b) = (1.0, 2.0, 3.0)$, that should be $e^{2} \approx 7.389$.

In [74]:
f.derivative_at(a, {a: 1.0, b: 2.0, c: 3.0})

-4.0
-1.1455000338086134


13.893553038339718

Let $g(x, y) = x + e^{y-1}$, at $(x, y) = (1.0, 2.0)$, that should be $1+e \approx 3.718$.

In [75]:
x = Variable()
y = Variable()
g = x + exp(y-1)

In [76]:
g.evaluation_at({x: 1.0, y: 2.0})

3.718281828459045

$\dfrac{\partial g}{\partial x} = 1$

In [77]:
g.derivative_at(x, {x: 1.0, y: 2.0})

1.0

$\dfrac{\partial g}{\partial y} = e^{y-1}$, at $(x, y) = (1.0, 2.0)$, that should be $e^{1} \approx 2.718$.

In [78]:
g.derivative_at(y, {x: 1.0, y: 2.0})

2.718281828459045

#### Test for Multiple

In [79]:
x = Variable()
y = Variable()
g = x*y
f = x/y

In [80]:
g.evaluation_at({x: 2.0, y: 2.0})

4.0

In [81]:
g.derivative_at(x, {x: 1.0, y: 2.0})

2.0

In [82]:
f.evaluation_at({x: 3.0, y: 2.0})

1.5

In [53]:
f.derivative_at(y, {x: 4.0, y: 2.0})

1.0

In [54]:
a = 3.4

In [24]:
2/3

0.6666666666666666